In [8]:
import Pkg; 
Pkg.add("LinearMaps")
Pkg.add("IterativeSolvers")

   Resolving package versions...
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
   Installed IterativeSolvers ─ v0.9.2
    Updating `C:\Users\zinli\.julia\environments\v1.8\Project.toml`
  [42fd0dbc] + IterativeSolvers v0.9.2
    Updating `C:\Users\zinli\.julia\environments\v1.8\Manifest.toml`
  [42fd0dbc] + IterativeSolvers v0.9.2
Precompiling project...
  ✓ IterativeSolvers
  1 dependency successfully precompiled in 2 seconds. 93 already precompiled.


In [53]:
using LinearMaps
using IterativeSolvers
using FFTW
using Base.Threads
using PaddedViews
using BenchmarkTools
using ChainRulesCore
using Zygote
using LinearAlgebra
using FiniteDifferences

function g2g̃(g)
    g̃ = complex.(zeros(size(g)),zeros(size(g)))
    n = size(g)[3]
    Threads.@threads for i in 1:n
        g̃[:,:,i] .= fft(g[:,:,i])
    end
    g̃
end

function g2g̃ᵀ(g)
    g̃ᵀ = complex.(zeros(size(g)),zeros(size(g)))
    n = size(g)[3]
    Threads.@threads for i in 1:n
        g̃ᵀ[:,:,i] .= fft(reverse(g[:,:,i]))
    end
    g̃ᵀ
end

⊛(g̃,u) = real.(ifftshift( ifft( g̃ .* fft(u) ) ))
✪(v,u) = real.(ifftshift( ifft( fft(v) .* conj.(fft(u)))))
syz(u,g̃) = collect(sym_paddedviews(0,u,g̃)[1])
v∂gu(g̃,u,v) = ✪(syz(v,g̃),syz(u,g̃))

function conv(u,K̃,fwd::Bool)
    nᵢ, nₖ = size(u)[1], size(K̃)[1]
    nₒ = nₖ-nᵢ

    i₁ = fwd==true ? 1+(nᵢ÷2) : nᵢ÷2
    i₂ = i₁+nₒ-1
    ⊛(K̃,syz(u,K̃))[i₁:i₂,i₁:i₂]
end

function G(u,g̃)
    nᵢ, nₖ = size(u)[1], size(g̃)[1]
    nₒ = nₖ-nᵢ
    n = size(g̃)[3]
    Gu = zeros(Float64,nₒ,nₒ,n)
    Threads.@threads for i in 1:n
        Gu[:,:,i] .= conv(u[:,:,i],g̃[:,:,i],true)
    end

    sum(Gu,dims=3)[:,:,1]
end

function GᵀG(u,g̃,g̃ᵀ)
    
    Gu = G(u,g̃)
    n = size(g̃)[3]
    GᵀGu = zeros(size(u))
    Threads.@threads for i in 1:n
        GᵀGu[:,:,i] .= conv(Gu,g̃ᵀ[:,:,i],false)
    end

    GᵀGu
end

function y∂GᵀGx(g̃,x,y)

    Gx = G(x,g̃)
    Gy = G(y,g̃)

    n = size(g̃)[3]
    ∂g = zeros(size(g̃))
    Threads.@threads for i in 1:n
        ∂g[:,:,i] .= v∂gu(g̃[:,:,i],x[:,:,i],Gy) .+ v∂gu(g̃[:,:,i],y[:,:,i],Gx)
    end
    
    ∂g
end

GᵀGαI(x,g̃,g̃ᵀ,α) = GᵀG(x,g̃,g̃ᵀ) .+ α.*x

function regress(g̃,g̃ᵀ,α,y)

    dims = size(y)

    A(ξ) = vec(GᵀGαI(reshape(ξ,dims),g̃,g̃ᵀ,α))
    Â = LinearMap(A,size(vec(y))[1],issymmetric=true,isposdef=true)
    reshape(cg(Â,vec(y),reltol=10^(-16)),dims)

end

function G2Û!(g,α,U,g̃,g̃ᵀ)

    g̃[:,:,:] = g2g̃(g)[:,:,:]
    g̃ᵀ[:,:,:] = g2g̃ᵀ(g)[:,:,:]
    
    Û = similar(U)
    Threads.@threads for i in 1:size(U)[4]
        y = GᵀG(U[:,:,:,i],g̃,g̃ᵀ)
        Û[:,:,:,i] .= regress(g̃,g̃ᵀ,α,y)[:,:,:]
    end
    Û

end

function Û2G!(∇ℓ,g̃,g̃ᵀ,α,U,Û)

    (n1,n2,n3) = size(g̃)
    n4 = size(U)[4]
    ∂g = zeros(Float64,n1,n2,n3,n4)
    Threads.@threads for i in 1:size(Û)[4]
        Λ = regress(g̃,g̃ᵀ,α,∇ℓ[:,:,:,i])
        ∂g[:,:,:,i] .= y∂GᵀGx(g̃,U[:,:,:,i].-Û[:,:,:,i],Λ)[:,:,:]
    end
    sum(∂g,dims=4)[:,:,:,1]

end

function ChainRulesCore.rrule(::typeof(G2Û!), g,α,U,g̃,g̃ᵀ)
    Û = G2Û!(g,α,U,g̃,g̃ᵀ)
    function G2Û!_pullback(∇ℓ)

        ∂g = @thunk(Û2G!(∇ℓ,g̃,g̃ᵀ,α,U,Û))
        NoTangent(), ∂g, ZeroTangent(), ZeroTangent(), ZeroTangent(), ZeroTangent()

    end
    Û, G2Û!_pullback
end

function G2ℓ(g,U,α)

    g̃ = Array{ComplexF64,3}(undef,size(g))
    g̃ᵀ = Array{ComplexF64,3}(undef,size(g))
    Û = G2Û!(g,α,U,g̃,g̃ᵀ)

    ntrain = size(U)[4]
    ret = 0.0
    for i in 1:ntrain
        ret += norm(U[:,:,:,i].-Û[:,:,:,i])/norm(U[:,:,:,i])
    end

    ret/ntrain  
end

using Random
Random.seed!(1234)
ng = 8
nu = 6
n = 2
g = rand(ng,ng,n)
u = rand(nu,nu,n)
α = 10.0
ntrain = 2
U = rand(nu,nu,n,ntrain)

gdat = Zygote.gradient(x->G2ℓ(x,U,α),g)[1];
display(gdat)


8×8×2 Array{Float64, 3}:
[:, :, 1] =
  4.40457e-19   1.19656e-18   2.18873e-18  …   7.45389e-20  6.55254e-19
 -1.49078e-19  -0.00701483   -0.010351        -0.0011517    0.000272301
  1.0842e-18   -0.00477776   -0.00933638      -0.00413263   0.00162142
  1.68051e-18  -0.00424246    0.00205385      -0.00630301   0.0049771
  1.89735e-19  -0.0048948    -0.0088605        0.00925109   0.00452191
 -2.1684e-19   -0.00742128   -0.00738723   …   0.019347     0.0071445
 -1.6263e-18    0.00403822   -0.00236706       0.0132421    0.00634608
 -2.84603e-19   0.00524912    0.00597085       0.000862443  0.00349445

[:, :, 2] =
  9.99499e-19  -2.44575e-19   5.65818e-19  …   2.69356e-18   1.84409e-18
 -4.87891e-19  -0.0018998    -0.00365583      -0.00834321   -0.00925429
  5.42101e-20  -0.00149615   -0.00651668      -0.0197244    -0.0244105
 -6.50521e-19  -0.00928567   -0.00637225      -0.0171399    -0.0210516
 -2.1684e-19   -0.00844865   -0.012275        -0.00995304   -0.0120947
  4.33681e-19  -0.005090

In [54]:
Δ = grad(central_fdm(5,1),x->G2ℓ(x,U,α),g)[1];
display(maximum(abs.(Δ .- gdat))/mean(abs.(Δ)))
display(Δ)

2.0796489883928402e-11

8×8×2 Array{Float64, 3}:
[:, :, 1] =
  2.80292e-14  -1.48526e-14   4.52768e-14  …   1.73202e-14  -2.9402e-15
  3.58807e-14  -0.00701483   -0.010351        -0.0011517     0.000272301
 -1.88352e-14  -0.00477776   -0.00933638      -0.00413263    0.00162142
 -2.00532e-14  -0.00424246    0.00205385      -0.00630301    0.0049771
 -5.60122e-14  -0.0048948    -0.0088605        0.00925109    0.00452191
  1.41752e-14  -0.00742128   -0.00738723   …   0.019347      0.0071445
  3.06829e-14   0.00403822   -0.00236706       0.0132421     0.00634608
  2.77904e-14   0.00524912    0.00597085       0.000862443   0.00349445

[:, :, 2] =
  4.09037e-14   0.0          3.54742e-14  …   0.0           0.0
  3.02797e-14  -0.0018998   -0.00365583      -0.00834321   -0.00925429
  4.42153e-14  -0.00149615  -0.00651668      -0.0197244    -0.0244105
 -3.61541e-14  -0.00928567  -0.00637225      -0.0171399    -0.0210516
 -4.98677e-15  -0.00844865  -0.012275        -0.00995304   -0.0120947
  3.25427e-14  -0.00509024  -0

In [10]:
function G2û!(g,α,u,g̃,g̃ᵀ)

    g̃[:,:,:] = g2g̃(g)[:,:,:]
    g̃ᵀ[:,:,:] = g2g̃ᵀ(g)[:,:,:]
    y = GᵀG(u,g̃,g̃ᵀ)
    û = regress(g̃,g̃ᵀ,α,y)

    û
end

function û2G!(∇ℓ,g̃,g̃ᵀ,α,u,û)

    Λ = regress(g̃,g̃ᵀ,α,∇ℓ)

    y∂GᵀGx(g̃,u.-û,Λ)

end

function ChainRulesCore.rrule(::typeof(G2û!), g,α,u,g̃,g̃ᵀ)
    û = G2û!(g,α,u,g̃,g̃ᵀ)
    function G2û!_pullback(∇ℓ)

        ∂û = @thunk(û2G!(∇ℓ,g̃,g̃ᵀ,α,u,û))
        NoTangent(), ∂û, ZeroTangent(), ZeroTangent(), ZeroTangent(), ZeroTangent()

    end
    û, G2û!_pullback
end

function G2ℓ!(g,U,α)

    g̃ = Array{ComplexF64,3}(undef,size(g))
    g̃ᵀ = Array{ComplexF64,3}(undef,size(g))

    ntrain = size(U)[4]
    ret = 0.0
    for i in 1:ntrain
        û =  G2û!(g,α,U[:,:,:,i],g̃,g̃ᵀ)
        ret += norm(U[:,:,:,i].-û)/norm(U[:,:,:,i])
    end

    ret/ntrain  
end

ng = 8
nu = 4
n = 2
g = rand(ng,ng,n)
u = rand(nu,nu,n)
α = 10.0
ntrain = 2
U = rand(nu,nu,n,ntrain)

gdat = Zygote.gradient(x->G2ℓ!(x,U,α),g)[1];
Δ = grad(central_fdm(5,1),x->G2ℓ!(x,U,α),g)[1];
display(maximum(abs.(Δ .- gdat))/mean(abs.(Δ)))

4.041212136722791e-11

In [2]:
# function yGᵀGx(x,g,y)
#     g̃ = g2g̃(g)
#     g̃ᵀ = g2g̃ᵀ(g)
#     sum(y.*GᵀG(x,g̃,g̃ᵀ))
# end

# ng = 6
# n = 1
# nu = 2
# nv = ng - nu
# g = rand(ng,ng,n)
# u = rand(nu,nu,n)
# v = rand(nv,nv,n)
# α = 10.0
# ntrain = 2
# U = rand(nu,nu,n,ntrain)

# x = rand(nu,nu,n)
# y = rand(nu,nu,n)
# g̃,g̃ᵀ = g2g̃(g), g2g̃ᵀ(g);
# gdat = y∂GᵀGx(g̃,x,y);
# Δ = grad(central_fdm(5,1),Γ->yGᵀGx(x,Γ,y),g)[1];
# display(maximum(abs.(Δ .- gdat))/mean(abs.(Δ)))
# a, b = rand(1), rand(1)
# tmp = GᵀGαI(a.*x + b.*y, g̃,g̃ᵀ,α) 
# tmp1 = a.*GᵀGαI(x,g̃,g̃ᵀ,α)
# tmp2 = b.*GᵀGαI(y,g̃,g̃ᵀ,α)
# display(maximum(abs.(tmp .- tmp1 .- tmp2)))

# dims = size(x)
# A(ξ) = vec(GᵀGαI(reshape(ξ,dims),g̃,g̃ᵀ,α))
# tmp = A(a.*vec(x) + b.*vec(y)) .- a.*A(vec(x)) - b.*A(vec(y))
# display(maximum(abs.(tmp)))

function regress(g̃,g̃ᵀ,α,y)

    dims = size(y)

    A(ξ) = vec(GᵀGαI(reshape(ξ,dims),g̃,g̃ᵀ,α))
    Â = LinearMap(A,size(vec(y))[1],issymmetric=true,isposdef=true)
    reshape(cg(Â,vec(y),reltol=10^(-16)),dims)

end

function G2û(g,α,u)

    g̃ = g2g̃(g)
    g̃ᵀ= g2g̃ᵀ(g)
    y = GᵀG(u,g̃,g̃ᵀ)
    û = regress(g̃,g̃ᵀ,α,y)

    û
end

function û2G(∇ℓ,g,α,u,û)

    g̃ = g2g̃(g)
    g̃ᵀ= g2g̃ᵀ(g)
    Λ = regress(g̃,g̃ᵀ,α,∇ℓ)

    y∂GᵀGx(g̃,u.-û,Λ)

end

function ChainRulesCore.rrule(::typeof(G2û), g,α,u)
    û = G2û(g,α,u)
    function G2û_pullback(∇ℓ)

        ∂û = @thunk(û2G(∇ℓ,g,α,u,û))
        NoTangent(), ∂û, ZeroTangent(), ZeroTangent()

    end
    û, G2û_pullback
end

function G2ℓ(g,α,u)

    û =  G2û(g,α,u)
    norm(u.-û)/norm(u)
  
end

ng = 8
nu = 4
n = 2
g = rand(ng,ng,n)
u = rand(nu,nu,n)
α = 10.0
gdat = Zygote.gradient(x->G2ℓ(x,α,u),g)[1];
Δ = grad(central_fdm(5,1),x->G2ℓ(x,α,u),g)[1];
display(maximum(abs.(Δ .- gdat))/mean(abs.(Δ)))


5.111066082286911e-11

In [2]:
function regress(g̃,g̃ᵀ,α,y)

    dims = size(y)

    A(ξ) = vec(GᵀGαI(reshape(ξ,dims),g̃,g̃ᵀ,α))
    Â = LinearMap(A,size(vec(y))[1],issymmetric=true,isposdef=true)
    reshape(cg(Â,vec(y),reltol=10^(-16)),dims)

end

function G2û!(g,α,u,g̃,g̃ᵀ)

    g̃[:,:,:] = g2g̃(g)[:,:,:]
    g̃ᵀ[:,:,:] = g2g̃ᵀ(g)[:,:,:]
    y = GᵀG(u,g̃,g̃ᵀ)
    û = regress(g̃,g̃ᵀ,α,y)

    û
end

function û2G!(∇ℓ,g̃,g̃ᵀ,α,u,û)

    Λ = regress(g̃,g̃ᵀ,α,∇ℓ)

    y∂GᵀGx(g̃,u.-û,Λ)

end

function ChainRulesCore.rrule(::typeof(G2û!), g,α,u,g̃,g̃ᵀ)
    û = G2û!(g,α,u,g̃,g̃ᵀ)
    function G2û!_pullback(∇ℓ)

        ∂û = @thunk(û2G!(∇ℓ,g̃,g̃ᵀ,α,u,û))
        NoTangent(), ∂û, ZeroTangent(), ZeroTangent(), ZeroTangent(), ZeroTangent()

    end
    û, G2û!_pullback
end

function G2ℓ!(g,α,u)

    g̃ = complex.(zeros(size(g)),zeros(size(g)))
    g̃ᵀ = complex.(zeros(size(g)),zeros(size(g)))
    û =  G2û!(g,α,u,g̃,g̃ᵀ)
    norm(u.-û)/norm(u)
  
end

ng = 8
nu = 4
n = 2
g = rand(ng,ng,n)
u = rand(nu,nu,n)
α = 10.0
gdat = Zygote.gradient(x->G2ℓ!(x,α,u),g)[1];
Δ = grad(central_fdm(5,1),x->G2ℓ!(x,α,u),g)[1];
display(maximum(abs.(Δ .- gdat))/mean(abs.(Δ)))

1.0503185845704046e-10

In [31]:
a = rand(3,3)
dims = size(a)
display(a)
a = vec(a)
display(a)
a = reshape(a,dims)

3×3 Matrix{Float64}:
 0.61014   0.142683  0.442251
 0.82297   0.61583   0.820144
 0.638666  0.148343  0.207111

9-element Vector{Float64}:
 0.6101398049155026
 0.8229698999016793
 0.6386662590420531
 0.14268287115596867
 0.6158302059999865
 0.14834304962597256
 0.44225118960331444
 0.8201439526840497
 0.20711123005888388

3×3 Matrix{Float64}:
 0.61014   0.142683  0.442251
 0.82297   0.61583   0.820144
 0.638666  0.148343  0.207111

In [50]:
rand(1)

1-element Vector{Float64}:
 0.3144782679242779